## First we import the necessary packages:

In [1]:
import dask.dataframe as dd
from pathlib import Path

## Define Datapaths

In [3]:
s0_path = Path("/workspaces/pearlModel/out/S0_msm_2024-08-05/parquet_output")
s3_path = Path("/workspaces/pearlModel/out/S3_msm_2024-08-05/parquet_output")

## First let's load and peek our dataframes.
## We take advantage of wildcard syntax to access all of our msm_black_male parquet files in parrallel

In [5]:
s0 = dd.read_parquet(s0_path/"msm_black_male"/"*"/"population.parquet")
s3 = dd.read_parquet(s3_path/"msm_black_male"/"*"/"population.parquet")

## What happens when we reference a dataframe?

In [7]:
s0

,age,age_cat,anx,ckd,delta_bmi,dm,dpr,esld,h1yy,hcv,ht,init_age,init_sqrtcd4n,intercept,last_h1yy,last_init_sqrtcd4n,lipid,ltfu_year,malig,mi,mm,n_lost,post_art_bmi,pre_art_bmi,return_year,smoking,sqrtcd4n_exit,status,t_anx,t_ckd,t_dm,t_dpr,t_esld,t_hcv,t_ht,t_lipid,t_malig,t_mi,t_smoking,time_varying_sqrtcd4n,year,year_died,years_out,become_obese_postART,bmiInt_eligible,bmiInt_impacted,bmiInt_ineligible_dm,bmiInt_ineligible_obese,bmiInt_ineligible_underweight,bmiInt_received,bmiInt_scenario,bmi_increase_postART,bmi_increase_postART_over5p,index,post_art_bmi_without_bmiInt,init_age_group
npartitions=20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int8,int8,bool,bool,float64,bool,bool,bool,int16,bool,bool,int8,float64,float64,int16,float64,bool,int16,bool,bool,int64,int32,float64,float64,int16,bool,float64,int8,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,float64,int16,float64,int16,bool,bool,bool,bool,bool,bool,bool,int8,bool,bool,float64,float64,int8
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Here is a peek at the data values themselves

In [8]:
s0.head()

,age,age_cat,anx,ckd,delta_bmi,dm,dpr,esld,h1yy,hcv,...,bmiInt_ineligible_dm,bmiInt_ineligible_obese,bmiInt_ineligible_underweight,bmiInt_received,bmiInt_scenario,bmi_increase_postART,bmi_increase_postART_over5p,index,post_art_bmi_without_bmiInt,init_age_group
id,,,,,,,,,,,,,,,,,,,,,
0,53,5,False,True,-0.179388,True,False,False,2009,False,...,False,False,False,False,0,False,False,0.0,0.0,1
1,50,5,False,True,6.932699,False,False,False,2005,False,...,False,False,False,False,0,False,False,0.0,0.0,0
2,52,5,True,True,1.742786,False,True,False,2005,False,...,False,False,False,False,0,False,False,0.0,0.0,0
3,53,5,False,False,0.431942,True,False,False,2007,False,...,False,False,False,False,0,False,False,0.0,0.0,1
4,49,4,False,False,-3.621221,False,True,False,2009,False,...,False,False,False,False,0,False,False,0.0,0.0,0


## What is I only wanted a subset of this data?

In [25]:
s0_subset = dd.read_parquet(s0_path/"msm_black_male"/"*"/"population.parquet", columns=["age", "anx", "ckd"])

In [12]:
s0_subset.head()

,age,anx,ckd
id,,,
0,53,False,True
1,50,False,True
2,52,True,True
3,53,False,False
4,49,False,False


## Let's show some calculations

In [26]:
s0_dm = dd.read_parquet(s0_path/"msm_black_male"/"*"/"population.parquet", columns=["dm", "age_cat"])
s3_dm = dd.read_parquet(s3_path/"msm_black_male"/"*"/"population.parquet", columns=["dm", "age_cat"])

In [28]:
s0_dm_risk = s0_dm.groupby('age_cat')['dm'].sum() / s0_dm.groupby('age_cat')['dm'].count()
s3_dm_risk = s3_dm.groupby('age_cat')['dm'].sum() / s3_dm.groupby('age_cat')['dm'].count()

In [30]:
risk_diff = s0_dm_risk - s3_dm_risk

In [31]:
risk_diff.compute()

age_cat
2    0.000603
3    0.000293
4    0.008918
5    0.010841
6    0.008263
7    0.005653
Name: dm, dtype: float64